# Autor: DIEGO SANZ VILLAFRUELA

## Algoritmia
### Práctica 9
En esta práctica se implementará algún método de *programación dinámica*.

Se pide la implementación de las clases y/o funciones que aparecen a continuación.

Las instrucción "pass" que aparecen en el cuerpo de las clases o funciones, se debe sustituir por la implementación adecuada.

Para cada clase o función que se pide se proporciona una o más funciones con algunos tests.

Al llamar a las funciones de test no debería saltar ninguna aserción.

### Subsecuencia común más larga

In [1]:
"""
    Dadas dos cadenas x e y devuelve una que es subsecuencia de ambas y que 
    tiene la longitud máxima de todas las subsecuencias comunes.
"""     
def subsecuencia_comun_mas_larga(X, Y):
    tamX = len(X)
    tamY = len(Y)
    # creamos una matriz vacia
    matrix = []
    for i in range(0,tamX):
        aux = []
        for j in range(0,tamY):
            aux.append(0)
        matrix.append(aux)
    
    # rellenamos la matriz
    subproblema(X,Y,tamX-1,tamY-1, matrix)
    # una vez rellenada, la recorremos para obtener la subsecuencia
    return backtrack(X,Y,tamX-1,tamY-1, matrix)

'''
    Obtener la subsecuencia común más larga
'''
def backtrack( X, Y, i, j, matriz):
    if i < 0 or j < 0:
        return ""
    elif  X[i] == Y[j]:
        return backtrack(X, Y, i - 1, j - 1, matriz) + X[i]
    else:
        if matriz[i][j-1] > matriz[i-1][j]:
            return backtrack(X, Y, i, j - 1, matriz)
        else:
            return backtrack(X, Y, i - 1, j, matriz) 

'''
    Rellenamos la matriz
'''
def subproblema(X,Y,i,j, matriz):
    if i < 0 or j < 0:
        return 0
    if matriz[i][j] != 0:
        return matriz[i][j]
    if X[i] == Y[j]:
        matriz[i][j] = subproblema(X,Y, i-1,j-1, matriz) + 1
    else:
        matriz[i][j] =  max ( subproblema(X,Y,i - 1, j,matriz), subproblema(X,Y, i , j - 1,matriz) )
    return matriz[i][j]



In [2]:
def test_subsecuencia_comun_mas_larga():
    """Casos de prueba para subsecuencia_comun_mas_larga."""
    
    for s1, s2, l in (
        ("GTTCCTAATA", "CGATAATTGAGA", 6),
        ("ACDAADDADDDDCCBCBCAD", "ADBDBBCDBDAABBDDDCBB", 11),
        ("BBDABCCADCCADADDCACAACBA", "DBCBBDCBADABBBCCCDCACAADDACADD", 17)
    ):    
        for x, y in ((s1, s2), (s2, s1)):
            sub = subsecuencia_comun_mas_larga(x, y)
            assert len(sub) == l
            for s in x, y:
                # comprobamos que sub es subsecuencia de s:
                it = iter(s)
                assert all(c in it for c in sub)
    
if __name__ == "__main__": 
    test_subsecuencia_comun_mas_larga()
    print("OK")    

OK


### Multiplicación encadenada de matrices
Dadas varias matrices, caracterizadas por sus dimensiones, se determinará qué multiplicaciones realizar.

In [25]:
"""
    Dadas las dimensiones de varias matrices a multiplicar, aplica el método
    de programación dinámica para para determinar en qué orden realizar las
    multiplicaciones.
    El número de matrices será la longitud de dimensiones menos uno.
    Las dimensiones de la matriz M_i están en las componentes i-1 e i de
    'dimensiones'.
    Devuelve el número de multiplicaciones de elementos a realizar y una
    cadena con la fórmula, incluyendo paréntesis (solo si son necesarios), en
    la que se realizarían las multiplicaciones.
    Por ejemplo '(M_1*(M_2*M_3))*M_4'.
"""
def multiplicacion_encadenada_matrices(dimensiones):

    tam = len(dimensiones) - 1
    matriz = [];minimos = [] # se utiliza minimos
    
    # se rellena la matriz
    for i in range(tam):
        matriz.append([])
        minimos.append([])
        for j in range(tam):
            matriz[i].append(None)
            minimos[i].append(None)
            if i == j:
                matriz[i][i] = 0
                

    #Algoritmo de los apuntes
    for d in range(1, tam):
        for i in range(1, tam - d + 1):
            j = i + d
            m = None; multiplicaciones = None
            for k in range(i, j):
                m_aux = matriz[i - 1][k - 1] + matriz[k][j - 1] + \
                    dimensiones[i - 1] * dimensiones[k] * dimensiones[j]
                if multiplicaciones == None or m_aux < multiplicaciones:
                    multiplicaciones = m_aux
                    m = k
            matriz[i - 1][j - 1] = multiplicaciones
            minimos[i - 1][j - 1] = m
            
    # Calcular la formula
    texto = calcular_formula(minimos, 1, tam)

    # Devolver los resultados
    return (matriz[0][tam - 1], texto)


def calcular_formula(km, i, j):
    
    if j == i:
        return "M_" + str(i)
    if j == i + 1:
        return "M_" + str(i) + "*" + "M_" + str(j)
    k = km[i - 1][j - 1]
    if k == i:
        return "M_" + str(i) + "*" + "(" + calcular_formula(km, k + 1, j) + ")"
    if k == j - 1:
        return "(" + calcular_formula(km, i, k) + ")" + "*" + "M_" + str(j)
    return "(" + calcular_formula(km, i, k) + ")" + "*" + "(" + calcular_formula(km, k + 1, j) + ")"

#print (multiplicacion_encadenada_matrices([13, 5, 89]) )

In [26]:
def test_orden_multiplicacion_matrices():
    """Casos de prueba para multiplicacion_encadenada_matrices."""
    
    for dimensiones, multiplicaciones, formula in (
        ([13, 5, 89], 13 * 5 * 89, "M_1*M_2"),
        ([13, 5, 89, 3, 34], 2856, "(M_1*(M_2*M_3))*M_4"), 
        ([2, 3, 5, 2, 4, 3], 78, "(M_1*(M_2*M_3))*(M_4*M_5)"),
        ([66, 87, 71, 43, 18, 19, 33, 98, 54], 498402, 
         "(M_1*(M_2*(M_3*M_4)))*(((M_5*M_6)*M_7)*M_8)")
    ):
        assert (multiplicacion_encadenada_matrices(dimensiones) ==
                (multiplicaciones, formula))   
    
if __name__ == "__main__": 
    test_orden_multiplicacion_matrices()
    print("OK")    

OK
